In [ ]:
# | default_exp _cli

In [ ]:
# | export

import asyncio
import multiprocessing
from typing import *

import typer

from fastkafka import _cli_docs, _cli_testing
from fastkafka._components.logger import get_logger
from fastkafka._server import run_fastkafka_server

In [ ]:
import os
import platform
import time

from typer.testing import CliRunner

from fastkafka._components.logger import suppress_timestamps
from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka._server import terminate_asyncio_process
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__, level=20)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

23-05-23 12:52:59.536 [INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")

In [ ]:
# | export


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        multiprocessing.cpu_count(),
        help="Number of FastKafka instances to run, defaults to number of CPU cores.",
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: str = typer.Option(
        "localhost",
        help="kafka_broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastaKafka class.",
    ),
) -> None:
    """
    Runs FastKafka application.

    Args:
        num_workers (int): Number of FastKafka instances to run, defaults to the number of CPU cores.
        app (str): Input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.
        kafka_broker (str): Kafka broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastKafka class.

    Raises:
        typer.Exit: If there is an unexpected internal error.
    """
    try:
        asyncio.run(
            run_fastkafka_server(
                num_workers=num_workers, app=app, kafka_broker=kafka_broker
            )
        )
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: run [OPTIONS] APP                                                                                          
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --num-workers               INTEGER  Number of FastKafka instances to run, defaults to number of CPU cores.     │
│                                      [default: 4]                                                               │
│ --kafka-broker              TEXT     kafka_broker, one of the keys of the kafka_brokers dictionary passed in    │
│                                      the constructor of FastaKafka class.                                       │
│                                      [default: localhost]                                                       │
│ --install-completion                 Install completion for the current shell.                                  │
│ --show-completion                    Show completion for the current shell, to copy it or customize the         │
│                                      installation.                                                              │
│ --help                               Show this message and exit.                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
async with ApacheKafkaBroker() as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:
        proc = await asyncio.create_subprocess_exec(
            "fastkafka",
            "run",
            "--num-workers",
            f"{2}",
            "--kafka-broker",
            "localhost",
            app,
            stdout=asyncio.subprocess.PIPE,
        )
        time.sleep(5)
        await terminate_asyncio_process(proc)
        outputs, errs = await proc.communicate()

        expected_returncode = 15 if platform.system() == "Windows" else 0
        assert proc.returncode == expected_returncode, outputs.decode("utf-8")

23-05-23 12:53:08.689 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-23 12:53:08.690 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-23 12:53:08.690 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-23 12:53:08.691 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-23 12:53:08.692 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
23-05-23 12:53:09.597 [INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
23-05-23 12:53:11.999 [INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
23-05-23 12:53:17.542 [INFO] fastkafka._server: terminate_asyncio_process(): Terminating the process 1582...
23-05-23 12:53:20.141 [INFO] fastkafka._server: terminate_asyncio_process(): Process 1582 terminated.
23-05-23 12:53:20.143 [INFO] fastkafka._components._subprocess: terminate_asyncio_pro

In [ ]:
# | export


_app.add_typer(_cli_docs._docs_app, name="docs")

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps", "--help"])

                                                                                                                   
 Usage: root docs install_deps [OPTIONS]                                                                           
                                                                                                                   

Installs dependencies for FastKafka documentation generation

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps"])
assert result.exit_code == 0

23-05-23 12:54:16.178 [INFO] fastkafka._components.docs_dependencies: AsyncAPI generator installed


In [ ]:
result = runner.invoke(_app, ["docs", "generate", "--help"])

                                                                                                                   
 Usage: root docs generate [OPTIONS] APP                                                                           
                                                                                                                   

Generates documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

23-05-23 12:55:36.144 [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmps5efh1ld/asyncapi/spec/asyncapi.yml' does not exist.
23-05-23 12:55:36.155 [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmps5efh1ld/asyncapi/spec/asyncapi.yml'
23-05-23 12:55:47.216 [INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
23-05-23 12:55:47.216 [INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmps5efh1ld/asyncapi/docs.






In [ ]:
result = runner.invoke(_app, ["docs", "serve", "--help"])

                                                                                                                   
 Usage: root docs serve [OPTIONS] APP                                                                              
                                                                                                                   

Generates and serves documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT     root path under which documentation will be created [default: .]                    │
│ --bind             TEXT     Some info [default: 127.0.0.1]                                                      │
│ --port             INTEGER  Some info [default: 8000]                                                           │
│ --help                      Show this message and exit.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
        "fastkafka",
        "docs",
        "serve",
        "--port=48000",
        app,
        stdout=asyncio.subprocess.PIPE,
    )
    time.sleep(30)
    await terminate_asyncio_process(proc)
    outputs, errs = await proc.communicate()
    expected_returncode = 15 if platform.system() == "Windows" else 0
    assert proc.returncode == expected_returncode, outputs.decode("utf-8")

In [ ]:
# | export


_app.add_typer(_cli_testing._testing_app, name="testing")

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps", "--help"])

                                                                                                                   
 Usage: root testing install_deps [OPTIONS]                                                                        
                                                                                                                   

Installs dependencies for FastKafka app testing

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps"])
assert result.exit_code == 0